# Computation

In [1]:
#import libraries cell

import json
import time
import findspark
import pyspark

import numpy as np
import pandas as pd

from kafka                import KafkaProducer
from kafka.admin          import KafkaAdminClient, NewTopic
 
from pyspark.sql           import functions as F
from pyspark.sql           import SparkSession
from pyspark.streaming     import StreamingContext
from pyspark.sql.types     import StructField, StructType, DoubleType, IntegerType
from pyspark.sql.functions import from_json, col, when, count, struct, collect_list
from pyspark               import SparkConf, SparkContext
from pyspark.sql.types import StringType 

## Spark setup

In [2]:
#initialisation of spark from the packages folder
findspark.init('/usr/local/spark')

In [3]:
#start session - specify port, application name, and configuration settings.

spark = SparkSession.builder\
        .appName("Project_CosmicRays_Dashboard_application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
        .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
        .config("spark.ui.port", "4041")\
        .getOrCreate()

#        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
spark

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aidin/.ivy2/cache
The jars for the packages stored in: /home/aidin/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a9e2cb7b-1c44-40bf-8f0e-69ec92f04b2f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 822ms :: artifacts dl 23ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central i

22/09/14 21:29:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 21:29:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Kafka Setup

In [4]:
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'

producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)
kafka_admin = KafkaAdminClient(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [5]:
kafka_admin.list_topics()

['stream', 'topic_stream', 'results', '__consumer_offsets']

In [6]:
kafka_admin.delete_topics(['results'])

DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='results', error_code=0)])

In [7]:
kafka_admin.list_topics()

['stream', 'topic_stream', '__consumer_offsets']

In [8]:
results_topic = NewTopic(name='results', 
                       num_partitions=4, 
                       replication_factor=1)
kafka_admin.create_topics(new_topics=[results_topic])


TopicAlreadyExistsError: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='results', num_partitions=4, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='results', error_code=36, error_message="Topic 'results' is marked for deletion.")])'.

In [9]:
kafka_admin.list_topics()

['stream', 'topic_stream', '__consumer_offsets']

In [10]:
# input dataframe and its source. 
# Define subscription to 'stream' - one of the two topics in kafka

inputDF = spark.readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
        .option('subscribe', 'topic_stream')\
        .load()

inputDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
# scheme of the imput data
schema = StructType(
        [
                StructField("HEAD", StringType()),
                StructField("FPGA", StringType()),
                StructField("TDC_CHANNEL", StringType()),
                StructField("ORBIT_CNT", StringType()),
                StructField("BX_COUNTER", StringType()),
                StructField("TDC_MEAS", StringType())
        ]
    )

In [12]:
# convert inputDF to jsonDF

jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: string (nullable = true)
 |    |-- FPGA: string (nullable = true)
 |    |-- TDC_CHANNEL: string (nullable = true)
 |    |-- ORBIT_CNT: string (nullable = true)
 |    |-- BX_COUNTER: string (nullable = true)
 |    |-- TDC_MEAS: string (nullable = true)



In [13]:
flatDF = jsonDF.select("value.HEAD", 
                       "value.FPGA", 
                       "value.TDC_CHANNEL",
                       "value.ORBIT_CNT",
                       "value.BX_COUNTER",
                       "value.TDC_MEAS")

flatDF.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)



In [14]:
# data-cleansing
df = flatDF.where(col("HEAD")==2)

# division of dataframe between chambers
df = df.withColumn('CHAMBER',
        when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 0) \
       .when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 1) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 2) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 3) \
       .when(( col( 'FPGA' ) == 1 ) &                                  ( col( 'TDC_CHANNEL' ) == 128 ), 4) \
               .otherwise(None))                                                                           \
      .filter( col( 'CHAMBER' ).isNotNull() )

df.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)
 |-- CHAMBER: integer (nullable = true)



In [15]:
# scintillator time offset by Chamber
# TODO compute ABSOLUTETIME and DRIFTIME
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

In [16]:
msg_json = {
    'epoch_id' : 0,
    'hits'     : 0,
    'CH0' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
#             'histo_SC'    : {'bin_edges'  : [],
#                              'bin_counts' : []
#                            }
            },
    'CH1' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
#             'histo_SC'    : {'bin_edges'  : [],
#                              'bin_counts' : []
#                            }
            },
    'CH2' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
#             'histo_SC'    : {'bin_edges'  : [],
#                              'bin_counts' : []
#                            }
            },
    'CH3' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                            },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
#             'histo_SC'    : {'bin_edges'  : [],
#                              'bin_counts' : []
#                            }
            },
}


In [17]:
def batch_proc(df, epoch_id):   

    '''
    Function for the batch processing of the data.
    The processing consists in retrieving
    the following informations:
        1. total number of processed hits, 
           post-clensing (1 value per batch)
        2. total number of processed hits,
           post-clensing, per chamber (4 values per batch)
        3. histogram of the counts of active TDC_CHANNEL,
           per chamber (4 arrays per batch)
        4. histogram of the total number of active
           TDC_CHANNEL in each ORBIT_CNT, per chamber 
           (4 arrays per batch)
        5. histogram of the counts of active TDC_CHANNEL,
           per chamber, ONLY for those orbits with at least
           one scintillator signal in it (4 arrays per batch)
        6. histogram of the DRIFTIME, per chamber 
           (4 arrays per batch) TODO
    
    Inputs:
        - df: spark dataframe with the data
        - epoch_id: batch index    
    '''

    msg_json['epoch_id']    = epoch_id

    # total number of processed hits
    # post-cleansing
    total_hits = df.filter(col('CHAMBER') != 4)\
                   .count()


    # total number of processed hits,
    # post-clensing, per chamber
    df_counts = df.filter(col('CHAMBER') != 4) \
                  .groupBy(   'CHAMBER')       \
                  .count()                     \
                  .withColumnRenamed('count', 
                                     'ch_hits')


    # histogram of the counts of active
    # TDC_CHANNEL, per chamber.
    df_counts_ch = df.filter(col('CHAMBER') != 4)            \
                     .groupBy('TDC_CHANNEL', 'CHAMBER')      \
                     .count()                                \
                     .groupBy('CHAMBER')                     \
                     .agg(struct(collect_list('TDC_CHANNEL') \
                                       .alias(  'bin_edges'),
                                 collect_list(      'count') \
                                       .alias(     'counts'),
                                )                            \
                          .alias('histo_CH')
                         )


    # histogram of the total number of active
    # TDC_CHANNEL in each ORBIT_CNT per chamber
    df_counts_orb = df.filter(col('CHAMBER') != 4)                    \
                      .groupBy('ORBIT_CNT', 'CHAMBER')                \
                      .agg(F.countDistinct("TDC_CHANNEL"))            \
                      .groupBy('count(TDC_CHANNEL)', 'CHAMBER')       \
                      .count()                                        \
                      .groupBy('CHAMBER')                             \
                      .agg(struct(collect_list('count(TDC_CHANNEL)')  \
                                        .alias(         'bin_edges'),
                                  collect_list(             'count') \
                                        .alias(            'counts'),
                                 )                                    \
                           .alias('histo_ORB')
                          )


    # Histogram of the count of active TDC_CHANNEL,
    # per chamber, only for those orbits with
    # at least one scintillatorin it
#    list_scint = df.filter(col('CHAMBER') == 4)   \
#                   .select('ORBIT_CNT').ORBIT_CNT \
#                   .collect                       \
#                   .toList

#    list_scint = df.filter(col('CHAMBER') == 4)   \
#                   .select('ORBIT_CNT').toPandas()['ORBIT_CNT'].tolist()
#    print(list_scint)
#    df_counts_sc = df.filter(col('CHAMBER') != 4 )                   \
#              .where( col( 'ORBIT_CNT' ).isin( list_scint ) ) \
#              .groupBy(  'TDC_CHANNEL', 'CHAMBER'       )     \
#              .count()                                        \
#              .agg(struct(collect_list('TDC_CHANNEL')         \
#                                  .alias('bin_edges'),
#                            collect_list(    'count')         \
#                                  .alias(   'counts'),
#                         ).alias('histo_SC')
#                  )

    # dataframe with results joined
    df_res = df_counts.join(df_counts_ch,  on='chamber') \
                      .join(df_counts_orb, on='chamber') \
                      .sort('chamber')                   \
                      .collect()
    #.join(df_counts_sc,   on='chamber') \
                      
    msg_json['hits'] = total_hits
    for i in range(len(df_res)):
        msg_json[f'CH{i}']['total_hits']              = df_res[i]['ch_hits']
        msg_json[f'CH{i}']['histo_CH'][  'bin_edges'] = df_res[i]['histo_CH'][ 'bin_edges']
        msg_json[f'CH{i}']['histo_CH'][' bin_counts'] = df_res[i]['histo_CH'][    'counts']
        msg_json[f'CH{i}']['histo_ORB'][ 'bin_edges'] = df_res[i]['histo_ORB']['bin_edges']
        msg_json[f'CH{i}']['histo_ORB']['bin_counts'] = df_res[i]['histo_ORB'][   'counts']
#        msg_json[f'CH{ch}']['histo_SC'][  'bin_edges'] = df_sc[        ch]['histo_SC']['bin_edges']
#        msg_json[f'CH{ch}']['histo_SC'][ 'bin_counts'] = df_sc[        ch]['histo_SC']['counts']
    producer.send('results', json.dumps(msg_json).encode('utf-8'))
    producer.flush()
    pass

In [18]:
df.isStreaming

True

In [ ]:
df.writeStream\
    .foreachBatch(batch_proc)\
    .start()\
    .awaitTermination()

22/09/14 21:29:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ef2a75d7-3d0e-4f67-96de-9072070fcf52. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/09/14 21:29:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/09/14 21:40:56 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:40:56 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.


22/09/14 21:40:56 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.


22/09/14 21:40:56 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.


22/09/14 21:40:57 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.


...

22/09/14 21:56:05 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:06 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:07 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available

22/09/14 21:56:31 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:32 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:33 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available

22/09/14 21:56:57 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:58 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:56:59 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available

22/09/14 21:57:22 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:57:23 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:57:24 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available

22/09/14 21:57:49 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:57:50 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available.
22/09/14 21:57:51 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0-1, groupId=spark-kafka-source-e7b94c18-83a7-48b6-b4ca-aaf4123e5181-373237678-driver-0] Connection to node 0 (pd-master/10.67.22.162:9092) could not be established. Broker may not be available

In [62]:
spark.stop()

ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=pd-master:9092 <connecting> [IPv4 ('10.67.22.162', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=pd-master:9092 <connecting> [IPv4 ('10.67.22.162', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=pd-master:9092 <connecting> [IPv4 ('10.67.22.162', 9092)]> returned error 111. Disconnecting.
ERROR:kafka.conn:Connect attempt to <BrokerConnection node_id=0 host=pd-master:9092 <connecting> [IPv4 ('10.67.22.162', 9092)]> returned error 111. Disconnecting.
